In [4]:
import torch
import torchvision
import torch.multiprocessing
from torch import nn
from torch.utils.data import DataLoader, TensorDataset
from torchvision import datasets
from torchvision.transforms import ToTensor
import numpy as np
import pandas as pd
import tensorflow as tf
from torch.utils.data import Dataset
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import torch.nn.functional as fun

In [7]:
training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor()
)

test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor()
)

  0%|          | 0/26421880 [00:00<?, ?it/s]

Extracting data\FashionMNIST\raw\train-images-idx3-ubyte.gz to data\FashionMNIST\raw



  0%|          | 0/29515 [00:00<?, ?it/s]

Extracting data\FashionMNIST\raw\train-labels-idx1-ubyte.gz to data\FashionMNIST\raw



  0%|          | 0/4422102 [00:00<?, ?it/s]

Extracting data\FashionMNIST\raw\t10k-images-idx3-ubyte.gz to data\FashionMNIST\raw



  0%|          | 0/5148 [00:00<?, ?it/s]

Extracting data\FashionMNIST\raw\t10k-labels-idx1-ubyte.gz to data\FashionMNIST\raw



In [22]:
training_data


Dataset FashionMNIST
    Number of datapoints: 60000
    Root location: data
    Split: Train
    StandardTransform
Transform: ToTensor()

In [9]:
test_data

Dataset FashionMNIST
    Number of datapoints: 10000
    Root location: data
    Split: Test
    StandardTransform
Transform: ToTensor()

In [10]:
batch_size = 100
train_loader = DataLoader(training_data,batch_size=batch_size)
test_loader = DataLoader(test_data,batch_size=batch_size)

In [11]:
class LogisticReg(torch.nn.Module):

    def __init__(self,n_input,n_output):
        super(LogisticReg,self).__init__()
        self.linear = torch.nn.Linear(n_input,n_output)

    def forward(self,x):
        y_pred = torch.sigmoid(self.linear(x))
        return y_pred
    

In [16]:
n_input = 28*28
n_output = 10
log_reg = LogisticReg(n_input,n_output)


In [17]:
optimizer = torch.optim.SGD(log_reg.parameters(), lr=0.001)
criterion = torch.nn.CrossEntropyLoss()

In [19]:
epochs = 10
Loss = []
acc = []
for epoch in range(epochs):
    for i, (images, labels) in enumerate(train_loader):
        optimizer.zero_grad()
        outputs = log_reg(images.view(-1, 28*28))
        loss = criterion(outputs, labels)
        # Loss.append(loss.item())
        loss.backward()
        optimizer.step()
        if i % 100 == 0:
            loss, current = loss.item(), (i + 1) * len(images)
            print(f"loss: {loss:>7f}  [{current:>5d}/{len(train_loader):>5d}]")
    Loss.append(loss.item())
    correct = 0
    for images, labels in test_loader:
        outputs = log_reg(images.view(-1, 28*28))
        _, predicted = torch.max(outputs.data, 1)
        correct += (predicted == labels).sum()
    accuracy = 100 * (correct.item()) / len(test_data)
    acc.append(accuracy)
    print('Epoch: {}. Loss: {}. Accuracy: {}'.format(epoch, loss.item(), accuracy))

loss: 2.129301  [  100/  600]
loss: 2.122943  [10100/  600]
loss: 2.116212  [20100/  600]
loss: 2.094087  [30100/  600]
loss: 2.091829  [40100/  600]
loss: 2.079532  [50100/  600]
Epoch: 0. Loss: 2.0806353092193604. Accuracy: 62.67
loss: 2.089301  [  100/  600]
loss: 2.083323  [10100/  600]
loss: 2.080890  [20100/  600]
loss: 2.056229  [30100/  600]
loss: 2.056772  [40100/  600]
loss: 2.047020  [50100/  600]
Epoch: 1. Loss: 2.0494801998138428. Accuracy: 63.59
loss: 2.056729  [  100/  600]
loss: 2.050426  [10100/  600]
loss: 2.051543  [20100/  600]
loss: 2.025636  [30100/  600]
loss: 2.027632  [40100/  600]
loss: 2.020263  [50100/  600]
Epoch: 2. Loss: 2.0235908031463623. Accuracy: 64.16
loss: 2.029698  [  100/  600]
loss: 2.022805  [10100/  600]
loss: 2.026738  [20100/  600]
loss: 2.000527  [30100/  600]
loss: 2.003052  [40100/  600]
loss: 1.997753  [50100/  600]
Epoch: 3. Loss: 2.0016634464263916. Accuracy: 64.41
loss: 2.006927  [  100/  600]
loss: 1.999371  [10100/  600]
loss: 2.0054

In [20]:
print(f"loss : {Loss[-1]}")

loss : 1.9173293113708496


In [21]:
print(f"accuracy : {acc[-1]}")

accuracy : 65.44
